In [2]:
# Standard Includes
import numpy as np
import re
import nltk
import pandas as pd

In [3]:
class pre_process():
    def __init__(self, filename):
        self.filename = filename
    
    def get_data(self):
        """Get our data from some csv file. The file should be a csv.
        Each cell should be a string and each string should be one entire speech.
        Must be run prior to clean()."""
        # Load our csv as strings
        self.data = open(self.filename).read().split()
        return self.data
    
    def seperate_speeches(self):
        debate = self.data
        mod_list = []
        can1_list = []
        can2_list = []

        i_new = -1
        i_old = -1
        while i_new >= -len(debate):
            if debate[i_new] == 'LEHRER:' or debate[i_new] == 'CROWLEY:' or debate[i_new] == 'SCHIEFFER:' or debate[i_new] == 'HOLT:' or debate[i_new] == 'RADDATZ:'or debate[i_new] == 'WALLACE:':
                mod_list += debate[i_new:]
                del debate[i_new:i_old]
                i_old = i_new

            elif debate[i_new] == 'OBAMA:' or debate[i_new] == 'CLINTON:':
                can1_list += debate[i_new:]
                del debate[i_new:i_old]
                i_old = i_new

            elif debate[i_new] == 'ROMNEY:' or debate[i_new] == 'TRUMP:':
                can2_list += debate[i_new:]
                del debate[i_new:i_old]
                i_old = i_new

            else:
                pass
            i_new -= 1
        
        speeches = (can1_list, can2_list)
        
        can_aslists = []
        for j in range(2):
            can_aslists.append(' '.join(speeches[j]))
        return can_aslists
    
    def clean(self, data):
        """Takes in our data and strips all unnecessary characters."""
        # Characters to remove
        remove = re.compile(r"[\'\"\\\!\,\/\;\{\}\[\.\]]")
        remove_names = re.compile(r'(ROMNEY:) (OBAMA:) (LEHRER:) (CLINTON:) (SCHIEFFER:) (CRAWLEY:) ()')
        # List to store clean speeches
        removed_names = []
        clean_data = []
        # Loop over each speech and remove all regex excluded characters defined above in remove
        for speech in data:
            remove_char = remove.sub("", speech)
            removed_names.append(remove_char)
        for speech in removed_names:
            clean_speech = remove_names.sub("", speech)
            clean_data.append(clean_speech)

        return clean_data
    
    def remove_stop_words(self, clean_speeches):
        stop_words = nltk.corpus.stopwords.words('english')
        removed_words_speeches = []
        for speech in clean_speeches:
            removed_words_speeches.append(' '.join([word for word in speech.split() if word not in stop_words]))
        return removed_words_speeches
    
    

In [4]:
class sentiment_analysis():
    def __init__(self, data):
        self.data = data
    
    def read_good_bad(self, pos, neg):
        self.pos = np.loadtxt(pos, dtype = str,encoding="ISO-8859-1")
        self.neg = np.loadtxt(neg, dtype=str ,encoding="ISO-8859-1")
        return [list(self.pos), list(self.neg)]

    def sentiment_count(self, speech):
        good_count = [x for x in speech if x in good_bad[0]]
        bad_count = [x for x in speech if x in good_bad[1]]
        return (good_count, bad_count)

In [5]:
Speech_list = ['/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2012/Obama_Romeny_1.txt',
                '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2012/Obama_Romeny_2.txt',
                '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2012/Obama_Romeny_3.txt',
                '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2016/Clinton_Trump_1.txt',
                '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2016/Clinton_Trump_2.txt',
                '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Debate_Scrape/Debates/2016/Clinton_Trump_3.txt']

good_list = '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Pre-Processing and Sentiment Analysis/opinion-lexicon-English/positive-words.txt'
bad_list = '/Users/rileylatham/Downloads/CMSE202/CMSE202_Final/Pre-Processing and Sentiment Analysis/opinion-lexicon-English/negative-words.txt'


In [7]:

output = []
i = 0
j = 0
Debate_Date = ['10/3/2012', '10/16/2012', '10/22/2012', '9/26/2016', '10/9/2016', '10/19/2016']
for val in Speech_list:
    
    data = pre_process(val)

    y = data.get_data()

    r = data.seperate_speeches()

    x = data.clean(r)

    z = data.remove_stop_words(x)
    
    SA = sentiment_analysis(z)
    
    good_bad = SA.read_good_bad(good_list, bad_list)
    
    for q in range(len(z)):
        if i==0 or i==1 or i==2:
            if q==0:
                Candidate_Name = 'Obama'
                Affiliation = 'D'
                Election_Year = 2012
            else:
                Candidate_Name = 'Romney'
                Affiliation = 'R'
                Election_Year = 2012
        if i==3 or i==4 or i==5:
            if q==0:
                Candidate_Name = 'Clinton'
                Affiliation = 'D'
                Election_Year = 2016
            else:
                Candidate_Name = 'Trump'
                Affiliation = 'R'
                Election_Year = 2016
        
        y = SA.sentiment_count(z[q].split())
        good = len(y[0])
        bad = len(y[1])
        
        output.append([Election_Year, Debate_Date[j], Candidate_Name, Affiliation, good, bad, good+bad])
    i += 1
    j += 1

In [8]:
sentiment_12_16 = pd.DataFrame(output, columns=['Election_Year', 'Debate_Date', 'Candidate_Name', 'Affiliation', 'Positive_Words', 'Negative_Words', 'Total_Sentiment'])

In [10]:
sentiment_12_16.to_csv('SentimentData_12-16.csv')